In [66]:
from ray.rllib.agents import ppo
import ray
import torch as th
from ray.tune.logger import pretty_print
from ray.tune import tune, grid_search

/home/pedro.cotovio/Repos/nclustgen/venv/lib/python3.8/site-packages/redis/connection.py:72: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)


In [67]:
try:
    ray.init()
except RuntimeError:
    passgrid_search

In [2]:
import nclustenv
env = nclustenv.make(nclustenv.version.ENV_LIST[0])

In [20]:
import dgl
batch = dgl.batch([env.state.current, env.state.current])
batch.ndata[0]

{'col': tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, F